# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 73 new papers today
          28 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/27 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.09320


extracting tarball to tmp_2210.09320...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.09322


extracting tarball to tmp_2210.09322...

...

/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2210.10008/retrieval.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:118: LatexWarning: attempting recovering figure images/white_lc_fit.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/tmp/ipykernel_2273/4030337529.py:34: LatexWarning: 2210.10008 did not run properly
Could not find figure images/white_lc_fit.pdf
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2210.10023


extracting tarball to tmp_2210.10023...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.09930-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.09930) | **An Intermediate-field Fast Radio Burst Model and the Quasi-periodic  Oscillation**  |
|| Jie-Shuang Wang, <mark>Xinyu Li</mark>, Zigao Dai, Xuefeng Wu |
|*Appeared on*| *2022-10-19*|
|*Comments*| *5 pages, 1 figure, submitted. Comments welcome!*|
|**Abstract**| Quasi-periodic oscillation (QPO) signals are discovered in some fast radio bursts (FRBs) such as FRB 20191221A, as well as in the X-ray burst associated with the galactic FRB from SGR 1935+2154. We revisit the intermediate-field FRB model where the radio waves are generated as fast-magnetosonic waves through magnetic reconnection near the light cylinder. The current sheet in the magnetar wind is compressed by a low frequency pulse emitted from the inner magnetosphere to trigger magnetic reconnection. By incorporating the wave dynamics of the magnetosphere, we demonstrate how the FRB frequency, the single pulse width, and luminosity are determined by the period, magnetic field, QPO frequency and quake energetics of the magnetar. We find that this model can naturally and self-consistently interpret the X-ray/radio event from SGR 1935+2154 and the QPO in FRB 20191221A. It can also explain the observed wide energy range of repeating FRBs in a narrow bandwidth. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.09536-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.09536) | **A search for relativistic ejecta in a sample of ZTF broad-lined Type Ic  supernovae**  |
|| Alessandra Corsi, et al. -- incl., <mark>Frank J. Masci</mark> |
|*Appeared on*| *2022-10-19*|
|*Comments*| *28 pages, 11 figures, submitted to AAS journal*|
|**Abstract**| The dividing line between gamma-ray bursts (GRBs) and ordinary stripped-envelope core-collapse supernovae (SNe) is yet to be fully understood. Observationally mapping the variety of ejecta outcomes (ultra-relativistic, mildly-relativistic or non-relativistic) in SNe of Type Ic with broad lines (Ic-BL) can provide a key test to stellar explosion models. However, this requires large samples of the rare Ic-BL events with follow-up observations in the radio, where fast ejecta can be probed largely free of geometry and viewing angle effects. Here, we present the results of a radio (and X-ray) follow-up campaign of 16 SNe Ic-BL detected by the Zwicky Transient Facility (ZTF). Our radio campaign resulted in 4 counterpart detections and 12 deep upper limits. None of the events in our sample is as relativistic as SN 1998bw and we constrain the fraction of SN 1998bw-like explosions to $< 19\%$ (3$\sigma$ Gaussian equivalent), a factor of $\approx 2$ smaller than previously established. We exclude relativistic ejecta with radio luminosity densities in between $\approx 5\times10^{27}$ erg s$^{-1}$ Hz$^{-1}$ and $\approx 10^{29}$ erg s$^{-1}$ Hz$^{-1}$ at $t\gtrsim 20$ d since explosion for $\approx 60\%$ of the events in our sample. This shows that SNe Ic-BL similar to the GRB-associated SN 1998bw, SN 2003lw, SN 2010dh, or to the relativistic SN 2009bb and iPTF17cw, are rare. Our results also exclude an association of the SNe Ic-BL in our sample with largely off-axis GRBs with energies $E\gtrsim 10^{50}$ erg. The parameter space of SN2006aj-like events (faint and fast-peaking radio emission) is, on the other hand, left largely unconstrained and systematically exploring it represents a promising line of future research. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.09320-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.09320) | **Stellar feedback-regulated black hole growth: driving factors from  nuclear to halo scales**  |
|| <mark>Lindsey Byrne</mark>, et al. |
|*Appeared on*| *2022-10-19*|
|*Comments*| *21 pages, 9 figures, submitted to MNRAS*|
|**Abstract**| Several recent simulations of galaxy formation predict two main phases of supermassive black hole (BH) accretion: an early, highly intermittent phase (during which BHs are under-massive relative to local scaling relations), followed by a phase of accelerated growth. We investigate physical factors that drive the transition in BH accretion in cosmological zoom-in simulations from the FIRE project, ranging from dwarf galaxies to galaxies sufficiently massive to host luminous quasars. The simulations model multi-channel stellar feedback, but neglect AGN feedback. We show that multiple physical properties, including halo mass, galaxy stellar mass, and depth of the central gravitational potential correlate with accelerated BH fueling: constant thresholds in these properties are typically crossed within ~0.1 Hubble time of accelerated BH fueling. Black hole masses increase sharply when the stellar surface density in the inner 1 kpc crosses a threshold Sigma1 ~ 10^9.5 Msun/kpc^2, a characteristic value above which gravity prevents stellar feedback from ejecting gas, and similar to the value above which galaxies are observed to quench. We further show that accelerated BH growth correlates with the emergence of long-lived, thin gas disks, as well as with virialization of the inner circumgalactic medium. The halo mass Mh ~ 10^12 Msun and stellar mass Mstar ~ 10^10.5 Msun at which BH growth accelerates correspond to ~L* galaxies. The fact that stellar feedback becomes inefficient at ejecting gas from the nucleus above this mass scale may play an important role in explaining why AGN feedback appears to be most important in galaxies above ~L*. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.09987-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.09987) | **Discovery of one neutron star candidate from radial velocity monitoring**  |
|| Hailong Yuan, et al. -- incl., <mark>Yongheng Zhao</mark>, <mark>Jifeng Liu</mark>, <mark>Haotong Zhang</mark> |
|*Appeared on*| *2022-10-19*|
|*Comments*| *16 pages, 9 figures*|
|**Abstract**| We report the discovery of one possible neutron star binary ($P_{\rm orb} =$ 0.8666 day) by using the LAMOST low-resolution spectroscopic data. The visible companion is a late A-type dwarf ($T_{\rm eff} = 7900 \pm 200$ K; log$g$ $=$ 4.3$\pm$0.2; $M =$ 1.7$\pm$0.1 M$_{\odot}$; $R\ =\ 1.7\pm0.2$ R$_{\odot}$), at a distance of 1.11$\pm0.03$ kpc. No double-lined feature can be seen from the GTC/HORuS high-resolution spectra, thus the radial velocity variation indicates an invisible object hiding in the binary. The system's optical light curves show clear ellipsoidal variability, suggesting that the visible companion is tidal distorted. By fitting the multi-band light curves with the ELC and WD codes, we constrain the mass of the invisible star to be 1.1--1.3 M$_{\odot}$. Spectral disentangling shows no additional component with optical absorption spectra, supporting the system contains one compact object. No X-ray or UV emission are detected in the ROSAT archive observations. Therefore, we suspect the invisible object is more likely a neutron star rather than a white dwarf. Our finding suggests the ability of LAMOST spectroscopic survey to discover X-ray quiescent compact objects. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.10023-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.10023) | **An extended [CII] halo around a massive star-forming galaxy at z = 5.3**  |
|| T. S. Lambert, et al. -- incl., <mark>R. Herrera-Camus</mark> |
|*Appeared on*| *2022-10-19*|
|*Comments*| *8 pages, 6 figures. Accepted to be published in MNRAS*|
|**Abstract**| High-redshift observations are often biased towards massive and bright galaxies that are not necessarily representative of the full population. In order to accurately study galaxy evolution and mass assembly at these redshifts, observations of ``normal'' main sequence galaxies are required. Here we present Atacama Large Millimeter/Submillimeter Array (ALMA) 0.3" resolution observations of the [CII] emission line at 158$\mu$m of HZ7, a main sequence galaxy at $z=5.25$. Comparing to archival rest-frame UV observations taken by the Hubble Space Telescope (HST), we find strong evidence of the existence of extended [CII] emission, which we estimate to be twice the size of the rest-frame UV emission, yielding one of the first high-redshift objects where a clear signature of a [CII] ``Halo'' has been detected to date. For a matched S\'ersic profile with n = 1, we measured a [CII] effective radius of $0.50\pm 0.04$" (3.07$\pm 0.25$ kpc) and an average rest-frame UV effective radius of $0.2\pm0.04$" ($1.48\pm0.16$ kpc). The [CII] morphology and kinematics of the system suggest a merging event resulting in a non rotating disk system. This event could be responsible for the extended [CII] emission. Alternatively, some potential obscured emission could also explain the [CII] to UV size ratio. These results contribute to the growing consensus with respect to the existence of extended [CII] emission around galaxies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.09662-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.09662) | **The burst mode of accretion in massive star formation with stellar  inertia**  |
|| <mark>D. M.-A. Meyer</mark>, et al. -- incl., <mark>S.-Y. Liu</mark> |
|*Appeared on*| *2022-10-19*|
|*Comments*| *Accepted to MNRAS, see this https URL&utm_campaign=mnras&utm_medium=email*|
|**Abstract**| The burst mode of accretion in massive star formation is a scenario linking the initial gravitational collapse of parent pre-stellar cores to the properties of their gravitationally unstable discs and of their accretion-driven bursts. In this study, we present a series of high-resolution 3D radiation-hydrodynamics numerical simulations for young massive stars formed out of collapsing 100 Mo molecular cores spinning with several values of the ratio of rotational-to-gravitational energies beta=5%-9%. The models include the indirect gravitational potential caused by disc asymmetries. We find that this modifies the barycenter of the disc, causing significant excursions of the central star position, which we term stellar wobbling. The stellar wobbling slows down and protracts the development of gravitational instability in the disc, reducing the number and magnitude of the accretion-driven bursts undergone by the young massive stars, whose properties are in good agreement with that of the burst monitored from the massive protostar M17 MIR. Including stellar wobbling is therefore important for accurate modeling disc structures. Synthetic ALMA interferometric images in the millimeter waveband show that the outcomes of efficient gravitational instability such as spiral arms and gaseous clumps can be detected for as long as the disc is old enough and has already entered the burst mode of accretion. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.10008-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.10008) | **A transmission spectrum of the sub-Earth planet L98-59~b in 1.1-1.7  $μ$m**  |
|| Mario Damiano, et al. -- incl., <mark>Sebastian Zieba</mark>, <mark>Laura Kreidberg</mark>, <mark>Christina L. Hedges</mark> |
|*Appeared on*| *2022-10-19*|
|*Comments*| *17 pages, 5 figures, 7 tables, accepted for publication in AJ*|
|**Abstract**| With the increasing number of planets discovered by TESS, the atmospheric characterization of small exoplanets is accelerating. L98-59 is a M-dwarf hosting a multi-planet system, and so far, four small planets have been confirmed. The innermost planet b is $\sim15\%$ smaller and $\sim60\%$ lighter than Earth, and should thus have a predominantly rocky composition. The Hubble Space Telescope observed five primary transits of L98-59b in $1.1-1.7\ \mu$m, and here we report the data analysis and the resulting transmission spectrum of the planet. We measure the transit depths for each of the five transits and, by combination, we obtain a transmission spectrum with an overall precision of $\sim20$ ppm in for each of the 18 spectrophotometric channels. With this level of precision, the transmission spectrum does not show significant modulation, and is thus consistent with a planet without any atmosphere or a planet having an atmosphere and high-altitude clouds or haze. The scenarios involving an aerosol-free, H$_2$-dominated atmosphere with H$_2$O or CH$_4$ are inconsistent with the data. The transmission spectrum also disfavors, but does not rules out, an H$_2$O-dominated atmosphere without clouds. A spectral retrieval process suggests that an H$_2$-dominated atmosphere with HCN and clouds or haze may be the preferred solution, but this indication is non-conclusive. Future James Webb Space Telescope observations may find out the nature of the planet among the remaining viable scenarios. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure images/white_lc_fit.pdf</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.09621-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.09621) | **Dust extinction map of the Galactic plane based on the VVV survey data**  |
|| <mark>Miaomiao Zhang</mark>, Jouni Kainulainen |
|*Appeared on*| *2022-10-19*|
|*Comments*| *accepted by MNRAS, 37 pages*|
|**Abstract**| Dust extinction is one of the most reliable tracers of the gas distribution in the Milky Way. The near-infrared (NIR) Vista Variables in the Via Lactea (VVV) survey enables extinction mapping based on stellar photometry over a large area in the Galactic plane. We devise a novel extinction mapping approach, XPNICER, by bringing together VVV photometric catalogs, stellar parameter data from StarHorse catalogs, and previously published Xpercentile and PNICER extinction mapping techniques. We apply the approach to the VVV survey area, resulting in an extinction map that covers the Galactic disk between 295 and 350 degrees at longitude and -2 to 2 degrees at latitude, and the Galactic bulge between -10 and 5 degrees at latitude. The map has 30 arcseconds spatial resolution and it traces extinctions typically up to about 10-20 mag of visual extinction and maximally up to Av~30 mag. We compare our map to previous dust based maps, concluding that it provides a high-fidelity extinction-based map, especially in its ability to recover both the diffuse dust component of the Galaxy and moderately extincted giant molecular cloud regions. The map is especially useful as independent, extinction-based data on the Galactic dust distribution and applicable for a wide range of studies from individual molecular clouds to the studies of the Galactic stellar populations. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error [Errno 2] No such file or directory: 'gs'</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2210.09930.md
    + _build/html/tmp_2210.09930/./QPO_model.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\beq}{\begin{eqnarray}}$
$\newcommand{\eeq}{\end{eqnarray}}$
$\newcommand{\bq}{B_{\rm QED}}$
$\newcommand{\fa}{f_{A}}$
$\newcommand{\fac}{f_{A,2}}$
$\newcommand{\Gc}{\Gamma_2}$
$\newcommand{\ffrb}{f_{\rm FRB}}$
$\newcommand{\lc}{L_{\rm FRB,43}}$
$\newcommand{\nuc}{\nu_{\rm c}}$
$\newcommand{\ncc}{\nu_{\rm c,9}}$
$\newcommand{\lwc}{L_{\rm w,45}}$
$\newcommand{\muc}{\mu_{33}}$
$\newcommand{\Bc}{B_{ d,15}}$
$\newcommand{\lac}{L_{A,44}}$
$\newcommand{\Bfms}{B_{\rm FMS}}$
$\newcommand{\xl}{\textcolor{red}}$
$\newcommand{\js}{\textcolor{cyan}}$
$\begin{document}$
$\begin{CJK*}{UTF8}{gkai}$
$\title[An Intermediate-field FRB model and QPO]{{An Intermediate-field Fast Radio Burst Model and the Quasi-periodic Oscillation}$
$}$
$\author[Wang et al.]{Jie-Shuang Wang$
$^{1}\thanks{E-mail:jiesh.wang@gmail.com},$
$Xinyu Li$
$^{2,3},$
$Zigao Dai$
$^{4,5},$
$Xuefeng Wu$
$^6\^{1}Max-Planck-Institut f\"ur Kernphysik, Saupfercheckweg 1, D-69117 Heidelberg, Germany \^2Canadian Institute for Theoretical Astrophysics, 60 St George St, Toronto, ON M5R 2M8\^3Perimeter Institute for Theoretical Physics, 31 Caroline Street North, Waterloo, Ontario, Canada, N2L 2Y5\^4Department of Astronomy, University of Science and Technology of China, Hefei 230026, China\^5School of Astronomy and Space Science, Nanjing University, Nanjing 210023, China\^6Purple Mountain Observatory, Chinese Academy of Sciences, Nanjing 210023, China$
$}$
$\label{firstpage}$
$\pagerange{\pageref{firstpage}--\pageref{lastpage}}$
$\maketitle$
$\end{CJK*}$
$\begin{abstract}$
$Quasi-periodic oscillation (QPO) signals are discovered in some fast radio bursts (FRBs) such as FRB 20191221A, as well as in the X-ray burst associated with the galactic FRB from SGR 1935+2154.$
$We revisit the intermediate-field FRB model where the radio waves are generated as fast-magnetosonic waves through magnetic reconnection near the light cylinder.$
$The current sheet in the magnetar wind is compressed by a low frequency pulse emitted from the inner magnetosphere to trigger magnetic reconnection.$
$By incorporating the wave dynamics of the magnetosphere, we demonstrate how the FRB frequency, the single pulse width, and luminosity are determined by the period, magnetic field, QPO frequency and quake energetics of the magnetar.$
$We find that this model can naturally and self-consistently interpret the X-ray/radio event from SGR 1935+2154 and the QPO in FRB 20191221A.$
$It can also explain the observed wide energy range of repeating FRBs in a narrow bandwidth.$
$\end{abstract}$
$\begin{keywords}$
$fast radio bursts -- stars: magnetars -- radiation mechanisms: non-thermal -- MHD -- magnetic reconnection$
$\end{keywords}$
$\section{Introduction}$
$In recent years, the study of fast radio bursts (FRBs) has been greatly advanced by the progress in the observations.$
$New detections of the galactic events \citep{CHIME2020Naturmagnetar,Bochenek2020Natur}, burst polarisation \citep[e.g.][]{Michilli2018,Luo2020Natur}, burst morphology \citep[e.g.][]{Pleunis2021ApJ}, source periodicity \citep[e.g.][]{Chime2020NaturPeriod,Pastor-Marazuela2021Natur,CHIME2022Natur}, source activity \citep[e.g.][]{Li2021Natur} and host galaxies \citep[e.g.][]{Bassa2017ApJL,Bhandari2022AJ}, have led to constraints on both progenitor models and radiation mechanisms.$
$However, a complete theoretical understanding of fast radio bursts is still not available and requires more effort \citep[see][for recent reviews]{Zhang2020Natur,Xiao2021SCPMA}.$
$Highly magnetised compact objects are usually involved as the central engine of FRBs, such as magnetars, pulsars, and accreting black holes. Among them, the magnetar scenario has been confirmed by the detection of the galactic event \citep{CHIME2020Naturmagnetar,Bochenek2020Natur,Li2021NatAs,Mereghetti2020ApJL,Ridnaia2021NatAs,Tavani2021NatAs}.$
$Theoretically, the energy resource can come from the internal magnetic energy of magnetars \citep[e.g.][]{Popov2013,Katz2016,Beloborodov2017,Margalit2020ApJL} or the gravitational potential energy when a companion star is involved \citep[e.g.][]{Wang2016,Dai2016,Wang2018,Zhang2020ApJL,Dai2020ApJL,Most2022MNRAS}.$
$Currently, the mainstream FRB radiation mechanisms can be divided into two categories: the near-field (or close-in) and the far-field (or far-away) model.$
$The near-field model, mainly based on the coherent curvature radiation model near the magnetar surface \citep[e.g.][]{Kumar2017,Yang2018,Lu2020MNRAS,WangFY2020ApJ,Yang2020ApJL,WangWY2020ApJ}, can explain the complex temporal behavior of FRBs, while the bunching mechanism for the coherent curvature radiation is unclear.$
$The far-field model incorporates the synchrotron maser emission at the shock front far from the magnetosphere as the coherent radio emission mechanism  \citep[e.g.][]{Lyubarsky2014,Beloborodov2017,Margalit2018,Waxman2017,Metzger2019MNRAS,Beloborodov2020ApJ,Plotnikov2019,Margalit2020ApJL,Wu2020ApJL,Xiao2020ApJL}.$
$This mechanism has been demonstrated using kinetic plasma simulations \citep[e.g.][]{Plotnikov2019,Sironi2021PhRvL}.$
$However, it is difficult to explain the sub-second scale quasi-periodic oscillations (QPOs) from FRBs \citep{CHIME2022Natur,Pastor-Marazuela2022arXiv}.$
$It is also found that neither model can fully explain the X-ray/radio event from SGR 1935+2154 \citep{Wang2020ApJ}.$
$Recently, a new type of FRB model is proposed where an FRB is radiated as fast-magnetosonic (FMS) waves generated from violent magnetic reconnection triggered by a low-frequency pulse (LFP) compressing the current sheet \citep{Lyubarsky2020ApJ}.$
$As the coherent radio emission is produced near the light cylinder, we refer to it as the "intermediate-field" model.$
$Kinetic plasma simulations \citep{Philippov2019,Mahlmann2022ApJL} have successfully demonstrated the emission of coherent FMS waves through this mechanism.$
$It is also found that the high linear polarisation \citep{Lyubarsky2020ApJ} and the downward frequency drifting \citep{Mahlmann2022ApJL} can be well explained.$
$However, the predicted frequency is significantly lower than the observation of the SGR 1935+2154 event \citep{Wang2020ApJ}.$
$In this paper, we revisit the intermediate-field model and propose that the LFP is produced through nonlinear conversion of Alfv\'en waves, which further leads to the generation of QPOs.$
$In section \ref{sec:radiation_injection}, we briefly review the coherent radiation mechanism from magnetic reconnection.$
$In section \ref{sec:FRB_QPO}, we study the injection of energy through Alfv\'en waves, the wave dynamics in the magnetosphere, and the generation of FRBs and QPOs.$
$In section \ref{sec:explainQPO}, we apply our model to observations.$
$The conclusion and discussion are presented in section \ref{sec:conclusion}.$
$Throughout this paper we adopt the shorthand X=X_n\times 10^n to describe the normalization of quantity X in cgs units.$
$\vspace{-5pt}$
$\section{Reconnection Driven Coherent Radio Emission}\label{sec:radiation_injection}$
$Magnetic reconnection is a process of changing the magnetic topology when two oppositely directed magnetic field lines approach each other forming a current sheet at the centre.$
$The onset of magnetic reconnection is triggered when the current sheet becomes tearing or kink unstable and breaks into a self-similar chain of plasmoids extending down to the kinetic scale \citep{Uzdensky2010PhRvL}.$
$Fast magnetic reconnection proceeds when the plasmoids collide and merge into larger islands.$
$During the coalescence of plasmoids, magnetic energy is dissipated and FMS waves are produced.$
$In highly magnetized systems with magnetization parameter \sigma\equiv B^2/4\pi \rho c^2 \gg1, FMS waves have similar dispersion relation as vacuum electromagnetic waves \citep{Thompson1998,Li2019} and can convert to coherent radio waves \citep[e.g.][]{Lyubarsky2019,Philippov2019,Lyubarsky2020ApJ}.$
$\cite{Lyubarsky2020ApJ} proposed that the coherent emission of FRBs are GHz FMS waves generated by the plasmoid collision during magnetic reconnection.$
$In their picture, the current sheets in the magnetar wind are compressed by an external outgoing LFP and become tearing unstable under the perturbation, and magnetic reconnection is  initiated.$
$Coherent FMS waves are produced in the reconnection and can escape as fast radio bursts.$
$Kinetic simulations by \citet{Mahlmann2022ApJL} have confirmed this process and found that FMS waves takes only a small fraction of the reconnection energy.$
$The characteristic frequency of the coherent FMS waves is determined by the size of the plasmoid \lambda'_c. Note we use the primed symbols to mark parameters measured in the co-moving frame of the plasma throughout the paper.$
$In the plasma comoving frame, \omega'_c \approx c/ \lambda'_c.$
$The typical plasmoid size is found to be proportional to the width of the current sheet \lambda'_c=\varsigma a' with \varsigma\sim10-100 \citep{Philippov2019,Lyubarsky2020ApJ}, and the width depends on the Larmor radius a'=\varpi r_{\rm L} with \varpi being a few.$
$\cite{Lyubarsky2020ApJ} gave the expression of the plasmoid size by balancing the pressure and energy,$
$\begin{eqnarray}$
$    \lambda'_c=\sqrt{\frac{8\pi\beta_{\rm rec}\varpi}{e\sigma_T}}\frac{m_ec^2}{B'^{3/2}},$
$\end{eqnarray}$
$where \sigma_T is the Thomson scattering cross section, \beta_{\rm rec}\sim0.1 is the reconnection rate, B'=B/\Gamma is the magnetic field in the comoving frame and \Gamma is the bulk Lorentz factor.$
$Therefore, the characteristic frequency in the observer's frame is,$
$\beq$
$\nu_{\rm c}\equiv\Gamma\omega'_c/2\pi\approx 2.1 \eta_1^{-1} \Gamma^{-1/2} B_{8}^{3/2}~{\rm GHz},\label{Eq:nuc}$
$\eeq$
$where \eta\equiv\varsigma \varpi^{1/2}\beta_{\rm rec}^{1/2}.$
$This expression is confirmed numerically by \cite{Mahlmann2022ApJL} but with a slightly different expression of \eta.$
$\begin{figure}$
$    \centering$
$    \vspace{-5pt}$
$    \includegraphics[width=0.35\textwidth]{QPO_model.pdf}$
$    \caption{The schematic picture of the intermediate-field FRB model (not to scale):$
$    Alfv\'en waves propagate along field lines of R\gtrsim R_0, and become non-linear at R_{\rm m}, which leads to the conversion to an LFP.$
$    (The Alfv\'en waves and LFP are drawn with different styles and unrealistic wavelengths to make their presence clear.)$
$    The LFP compress the current sheet in the striped magnetar wind near the light cylinder.$
$    High-frequency FMS waves are generated by merging islands during the violent magnetic reconnection, which can escape as an FRB.}$
$    \label{fig:magnetar_model}$
$\end{figure}$
$\vspace{-5pt}$
$\section{Intermediate-field FRB models and QPOs}\label{sec:FRB_QPO}$
$\subsection{Energy injection via Alfv\'en waves}$
$For a magnetar with a surface dipole magnetic field B_d and radius R_*\approx10^6 cm, the total magnetic energy is E_{B}=B_d^2R_{*}^3/6\approx1.7\times10^{47}\Bc^2 erg.$
$The internal magnetic field B_* can be higher and we parameterize it by B_*= \zeta B_d with \zeta>1.$
$The internal magnetic energy can be released by a sudden starquake and crustal motion which are accompanied by the crustal shear oscillations \citep[e.g.][]{Thompson1995MNRAS,Thompson2001,Duncan1998ApJL}.$
$Such oscillations are proposed to explain the QPOs observed in magnetar X-ray flares \citep[e.g.][]{Israel2005,Strohmayer2005,Strohmayer2006ApJ}.$
$As the shear wave propagates in the crust, it shakes the anchored magnetic field lines, and launches Alfv\'en waves into the magnetosphere \citep{Thompson2001,Li2016,Thompson2017,Bransgrove2020}.$
$The Alfv\'en waves carry energy into the magnetosphere and propagate along the field lines,$
$as sketched in Fig.~\ref{fig:magnetar_model}.$
$The field-line equation is r=R\sin^2{\theta}, where R is the maximum distance of the field line to the magnetar center and \theta is the angle to the magnetic axis.$
$For field lines with R\gg R_*, the field-line length is l\simeq\pi R/2.$
$The generated Alfv\'en waves of frequency \fa in the magnetosphere can propagate along field lines with l\geq l_0=\pi R_0/2=c/\fa=1.9\times10^{10}\fa^{-1}~{\rm cm} as a perturbation.$
$Note that Alfv\'en waves travelling on field lines with l<l_0 may suffer strong nonlinear interaction and the dissipation mechanism is unclear. Thus we only consider magnetar quakes occurred near the magnetic polar region, so that magnetar oscillations can launch Alfv\'en waves into the magnetosphere before damping \citep[e.g.][]{Levin2007MNRAS,Hoven2012}.$
$We consider a sudden release of internal magnetic energy of E_q=h S B_*^2/8\pi in the crust with a crust thickness h\approx10^5 cm and an area S_q.$
$The magnetar quake can simultaneously launch a number of waves with different frequencies, that are of different damping time-scales \citep[e.g.][]{Levin2007MNRAS,Hoven2012,Gabler2016MNRAS}.$
$We assume that the Alfv\'en wave of frequency f_A is the dominated one, and a number (n_A) of waves at this frequency has been excited.$
$The energy carried by each wave is parameterized as \epsilon E_q\approx L_A/f_A with a power L_A and \epsilon<1/n_A.$
$The wave is launched from the magnetar surface within the same area,$
$\begin{eqnarray}$
$    S_q=2.5\times10^8\lac\fac^{-1}\Bc^{-2}\epsilon^{-1}\zeta^{-2} {\rm cm}^2.$
$\end{eqnarray}$
$The corresponding wave amplitude in the magnetosphere is {\delta B}=\sqrt{8\pi L_A/S_qc}.$
$Its relative amplitude to the background field near the magnetar surface is$
$\begin{eqnarray}$
$    \frac{\delta B}{B_d}\approx 0.02\fac^{1/2}\epsilon^{1/2}\zeta,$
$\end{eqnarray}$
$As the Alfv\'en wave propagates in the magnetosphere, its amplitude grows as {\delta B}/{B_d}\propto r^{3/2} \citep[e.g.][]{Kumar2020,Yuan2020}.$
$When the relative amplitude grows to {\delta B}/{ B}\approx1 \citep{Yuan2020,Yuan2022ApJ} at a distance of$
$\begin{eqnarray}$
$   R_{\rm m}\approx 1.4\times 10^7\fac^{-1/3}\epsilon^{-1/3}\zeta^{-2/3}~\rm{cm},$
$\end{eqnarray}$
$the Alfv\'en waves will become nonlinear and shear the magnetosphere.$
$The background magnetic field here can be estimated from a dipole B_{\rm m}=B_*R_*^3R_{\rm m}^{-3}.$
$Under the strong shear, the magnetosphere becomes unstable.$
$A large part of the Alfv\'en waves will be converted to FMS waves that propagates outwards radially in the form of a LFP \citep{Yuan2021} due to the interaction with the curved magnetic field line.$
$The remaining Alfv\'en waves will dissipate inside the magnetosphere by nonlinear instabilities \citep{Yuan2020} or absorbed by the magnetar \citep{Li2015}.$
$Self interactions among multiple Alfv\'en waves can enhance the wave dissipation through turbulence cascade, increasing fast wave conversion rate \citep{Li2019} or leading to fast dissipation \citep{Li2021}.$
$\vspace{-5pt}$
$\subsection{Generation of FRBs}$
$An illustration for the generation of FRBs is shown in Fig. \ref{fig:magnetar_model}.$
$During the propagation of the LFP at r>R_{\rm m}, its toroidal field follows rB_{\rm LFP } = B_{\rm LFP, m}R_{\rm m} with B_{\rm LFP, m}\approx B_{\rm m} \citep{Yuan2020,Yuan2022ApJ}.$
$As the energy contained in the LFP is higher than that of background field, the magnetosphere is distorted.$
$The poloidal field follows r^2 B_p= B_{\rm m}R_{\rm m}^{2} from the conservation of magnetic flux.$
$Thus the magnetosphere at R>R_{\rm m} can be treated as a split monopole \citep{Parfrey2013,Yuan2020,Yuan2022ApJ}.$
$When the LFP propagates beyond the light cylinder at R_L=c P/2\pi, where P is the magnetar rotation period, it enters the magnetar wind zone.$
$The toroidal field of the LFP and the poloidal field of the magnetosphere at the light cylinder are$
$\begin{eqnarray}$
$    B_{\rm LFP, L}=3.4\times10^8\Bc\fac^{2/3}\epsilon^{2/3}\zeta^{4/3}\left(\frac{P}{3 {\rm s}}\right)^{-1} \rm{G}\label{eq:bphi_LC};\B_{p,m}(R_{\rm L})=3.4\times10^5\Bc\fac^{1/3}\epsilon^{1/3}\zeta^{2/3}\left(\frac{P}{3 {\rm s}}\right)^{-2} \rm{G}.\label{eq:bpol_LC}$
$\end{eqnarray}$
$The LFP compresses the striped wind and accelerates the plasma to a bulk Lorentz factor of \Gamma\approx0.5\sqrt{B_{\rm LFP, L}/B_{\rm w}(R_{\rm L}) }, where the wind field is described as a monopole with B_{\rm w}(r)=B_{p}(R_L)R_L/r and B_{p}(R_L) is the poloidal field of the magnetosphere at the light cylinder.$
$The pulse front propagates into the magnetar wind launched by the unperturbed magnetosphere with B_{p}(R_L)=B_*R_*^3R_{\rm L}^{-3}.$
$However, the energy carried by the LFP amplifies the magnetic field at the light cylinder to B_{p,m}(R_{\rm L}) \citep{Yuan2020,Yuan2022ApJ}, which further enhances the toroidal magnetic field in the striped wind to B_{\rm w}(R_{\rm L})\sim B_{p,m}(R_{\rm L}).$
$Therefore, the rear part of LFP will interact with the current sheets in the striped wind launched by the perturbed magnetosphere (see. Fig. \ref{fig:magnetar_model}).$
$In reality, the amplification of the wind toroidal field is highly dynamic, we therefore expect the wind field to be in the range B_{\rm w}(R_{\rm L})\in[B_*R_*^3R_{\rm L}^{-3},\; B_{p,m}(R_{\rm L})].$
$The corresponding maximum and minimum Lorentz factors of the accelerated LFPs are then$
$\begin{eqnarray}$
$    &&\Gamma^{\rm max}=496 \epsilon^{1/3}\zeta^{2/3}\fac^{1/3}\left(\frac{P}{3 {\rm s}}\right);\\\nonumber$
$    &&\Gamma^{\rm min}=16 \epsilon^{1/6}\zeta^{1/3}\fac^{1/6}\left(\frac{P}{3 {\rm s}}\right)^{1/2}.\label{eq.Gamma}$
$\end{eqnarray}$
$In the following we parameterize \Gamma=100\Gc which lies between \Gamma^{\rm min} and \Gamma^{\rm max}.$
$And we expect \Gamma\rightarrow\Gamma^{\rm min} for LFPs consisting of multiple waves (e.g. n_A\gtrsim3), such as in the SGR 1935+2154 event and FRB 20191221A (see Sec. \ref{sec:explainQPO} for more details).$
$The LFP compresses the current sheet in the striped wind, which triggers magnetic reconnection and leads to the generation of high-frequency FMS waves \citep{Lyubarsky2020ApJ,Mahlmann2022ApJL}.$
$Substituting B_{\rm LFP,L} and \Gamma into Eq.~(\ref{Eq:nuc}), we obtain the frequency of the FMS waves,$
$\begin{eqnarray}$
$    \nuc=1.3\fac\Bc^{3/2}\epsilon\zeta^2\left(\frac{P}{3 {\rm s}}\right)^{-3/2}\eta_1^{-1}\Gc^{-1/2} \rm{GHz},\label{eq:nu_final}$
$\end{eqnarray}$
$which lies in the radio band.$
$The produced FMS waves suffers from nonlinear interactions in the magnetosphere that may prevent it from escaping.$
$We can estimate the optical depth of the nonlinear interactions for the FMS waves following \citet{Lyubarsky2020ApJ},$
$\begin{eqnarray}$
$    \tau_{\rm NL}\sim \frac{\kappa 2\pi\nu R_{\rm L}}{2\Gamma^2c}=0.5\kappa_{-5.5}\ncc\left(\frac{P}{3 {\rm s}}\right)\Gc^{-2},$
$\end{eqnarray}$
$where \kappa=\Bfms^2/B_{\rm LFP, L}^2 is the energy ratio of the high-frequency FMS waves to the LFP. In 2D simulations, it is found that a fraction \kappa\sim10^{-4} of energy from the LFP is converted to the FMS, interpreted as an upper limit of the efficiency \citep{Mahlmann2022ApJL}.$
$The ratio of radio to X-ray energy of FRB200428 is found to be 10^{-6}-10^{-5} \citep{Ridnaia2021NatAs}.$
$Here we take \kappa\sim10^{-5.5} to be the fiducial value.$
$Therefore, the FMS waves generated at the light cylinder can escape the magnetosphere without significant dissipation and be observed as FRBs.$
$There are also current sheets behind the LFP inside the magnetosphere, which has been suggested to produce an FRB \citep{Wang2020ApJ,Yuan2020,Yuan2022ApJ}.$
$However, we would argue below that the FMS waves produced behind the LFP cannot be the primary source of FRB.$
$On one hand, FMS waves emitted deep inside the magnetosphere suffers from strong dissipation due to the nonlinear interactions.$
$The optical depth inside the magnetosphere is \tau_{\rm NL}\approx({\Bfms}/{B_p})^2\omega R/c \citep[see Eq. A3 in][]{Lyubarsky2020ApJ} with the FMS wave magnetic field {\Bfms}=\sqrt{2L_{\rm FMS}/ r^2} and B_p being treated as a monopole.$
$Setting \tau_{\rm NL}\sim10, we obtain a mean-free-path$
$\begin{eqnarray}$
$\ell\sim6.1\times10^7\Bc^{1/2}\fac^{1/6}\epsilon^{1/6}\zeta^{1/3}\ncc^{-1/4}L_{\rm FMS,42}^{-1/4} {\rm cm},$
$\end{eqnarray}$
$which is much smaller than R_L for a typical magnetar with period 0.1-10~s.$
$On the other hand, the reconnection of the current sheet behind the LFP is supported by the reversed poloidal magnetic field \citep{Yuan2022ApJ}.$
$Near the light cylinder where the produced FMS can escape, we can estimate$
$\begin{eqnarray}$
$    \frac{B_{p,m}^2(R_{\rm L})}{B_{\rm LFP, L}^2}=1.0\times10^{-6}\fac^{-2/3}\epsilon^{-2/3}\zeta^{-4/3}\left(\frac{P}{3 {\rm s}}\right)^{-2}.$
$\end{eqnarray}$
$The magnetic energy here is much smaller than the energy of the LFP, therefore the escaping FMS wave must be dominated by those generated in the striped wind.$
$\vspace{-5pt}$
$\subsection{QPOs in FRBs}$
$The LFP is made up of low-frequency FMS waves converted from the propagating Alfv\'en waves.$
$This process can be viewed as the nonlinear interaction between Alfv\'en waves and the curved background magnetic field.$
$The leading order wave interaction is the three-wave interaction with the frequency of FMS wave given by f_{\rm LFP}=f_A+f_{\rm bg}.$
$As the background magnetic field can be viewed as a large scale perturbation, i.e. f_{\rm bg}\approx 0, the resulting LFP wave will have the same frequency as the Alfv\'en wave f_{\rm LFP}\approx \fa.$
$The LFP propagates outward radially with its thickness conserved as found by numerical simulations \citep{Yuan2022ApJ}. Therefore the LFP frequency remains constant.$
$The energy dissipation of the magnetic reconnection at the light cylinder will be mediated by this LFP wave frequency, so that we expect$
$\begin{eqnarray}$
$    \ffrb\sim f_{\rm LFP}\approx\fa.$
$\end{eqnarray}$
$Note in this expression, we assume \fa>1/P, which will in general be satisfied for magnetars with P\gtrsim0.1 s. Otherwise the FRB profile will also be modulated by the spin period.$
$The LFP accelerates during the propagation to the light cylinder.$
$Therefore the observed width of one single pulse and peak luminosity is$
$\begin{eqnarray}$
$    W_{\rm p}\approx1/(\Gamma\fa),\label{eq:tpulse}\L_{\rm FMS}\approx\kappa L_A \Gamma\label{eq:lfrb}.$
$\end{eqnarray}$
$The frequency of Alfv\'en waves depends on the magnetar properties.$
$\cite{Duncan1998ApJL} studied the shear wave in the magnetar crust and found that the frequency of toroidal modes with the lowest radial node (symbolized by _lt_0) follows \fa(_lt_0)\approx12.2l^{1/2}(l+1)^{1/2} \rm{Hz} with l\geq2 being an integer \citep{Duncan1998ApJL}.$
$Later it is found that the crustal shear modes would couple with the MHD modes of the magnetar core, and the frequency and duration of the oscillation will be modified \citep[e.g.][]{Levin2007MNRAS,Hoven2012,Gabler2016MNRAS}.$
$Such theories indicate that the fundamental frequency of magnetar QPOs can be as low as a few Hz, and it can be much higher for high order overtones, even up to a few kHz \citep[e.g.][]{Hoven2012}.$
$The observed QPO signals in FRBs will be dominated by one or several major frequencies which carries most of the energy, and the FRB light curve is made by their superposition.$
$\vspace{-5pt}$
$\section{Application to the observed QPOs and the SGR 1935+2154 event}\label{sec:explainQPO}$
$Observations show that the FRB from the galactic magnetar, SGR 1935+2154 \citep{CHIME2020Naturmagnetar,Bochenek2020Natur}, is associated with a hard X-ray burst by several X-ray instruments\citep{Li2021NatAs,Mereghetti2020ApJL,Ridnaia2021NatAs,Tavani2021NatAs}.$
$This magnetar is of spin period and surface magnetic field P=3.2 s and B_*=2.2\times10^{14} G respectively \citep{Olausen2014,Israel2016}.$
$Two radio pulses separated by 30 ms are found in this FRB, while more pulses are detected in X-ray bands.$
$Especially, a QPO of \fa\approx40 Hz and n_A\gtrsim8 is detected in the X-ray light curve by the Insight-HXMT \citep{Li2022ApJ}.$
$The observed pulse width is 0.3-0.6 ms ms by CHIME and STARE2 in 0.6-1.3 GHz \citep{CHIME2020Naturmagnetar,Bochenek2020Natur}.$
$Substituting \epsilon<1/n_A\sim1/8, W_{\rm p}\sim0.5 ms, and  \nuc\sim1 GHz as well as the observed radio peak luminosity L_{\rm FMS}\approx10^{38} erg/s \citep{Bochenek2020Natur,Zhou2020ApJ} into Eqs. (\ref{eq:nu_final}, \ref{eq:tpulse} and \ref{eq:lfrb}), we obtain \Gamma\sim50, \zeta\gtrsim11\eta_1^{1/2} and L_A\approx 6\times10^{41} \kappa_{-5.5}^{-1} erg/s.$
$The required \Gamma is in the allowed range [22\eta_1^{1/6}, 977\eta_1^{1/3}] from Eq. (\ref{eq.Gamma}).$
$And the required L_A is higher than the observed X-ray luminosity, 1.1\times10^{41}~erg/s \citep{Ridnaia2021NatAs}.$
$Therefore, our model can explain this event self-consistently.$
$QPO signals are also observed in individual FRB events, such as the 4.6 Hz QPO in FRB 20191221A, 357.1 Hz in FRB 20210206A, 93.4 Hz in FRB 20210213A detected by CHIME \citep{CHIME2022Natur}, and the 2.4 kHz QPO in FRB 20201020A detected by Apertif \citep{Pastor-Marazuela2022arXiv}.$
$The observed QPOs with frequencies larger than 93.4 Hz mentioned above can be explained by the overtones in a wide parameter range. While the 4.6 Hz QPO in FRB 20191221A may relate to the fundamental mode. We here focus on this source, which has the highest significance \citep{CHIME2022Natur}.$
$The observed radio frequency, single pulse width, and QPO frequency of FRB 20191221A are \nuc\approx0.7 GHz, W_{\rm p}\approx4 ms, and f_A\approx4.6 Hz with n_A\approx9, respectively.$
$The total duration of this event is \sim3 s and we would expect P>3 s so that the QPO signal is not contaminated by the magnetar rotation.$
$Substituting them into Eqs. (\ref{eq:nu_final} and \ref{eq:tpulse}), we obtain \Gamma=54 and \zeta\gtrsim9\eta_1^{1/2}(P/3 {\rm s})^{3/4}\Bc^{-3/4}.$
$The required \Gamma is in the allowed range [14\eta_1^{1/6}(P/3 {\rm s})^{3/4}\Bc^{-1/4},367\eta_1^{1/3}(P/3 {\rm s})^{3/2}\Bc^{-1/2}].$
$Thus the QPO in FRB 20191221A can also be explained by our model.$
$\vspace{-5pt}$
$\section{Conclusion and Discussion}\label{sec:conclusion}$
$In this paper we revisited the intermediate-field FRB model and studied the QPOs.$
$The launch of Alfv\'en waves into the magnetosphere generates an LFP, which is made up of FMS waves at approximately the same frequencies of Alfv\'en waves (\fa).$
$This further generates FRBs when the LFP dissipates its energy at the current sheet near the light cylinder.$
$The FRB light curve is thus modulated by the frequency of the LFP, exhibiting QPOs at a frequency \sim\fa.$
$The major difference of our calculations from \cite{Lyubarsky2020ApJ} is the treatment of magnetic field configurations of the LFP and the magnetar wind, as we considered the wave dynamics in the magnetosphere.$
$The FRB frequency (Eq. \ref{eq:nu_final}), the single pulse width (Eq. \ref{eq:tpulse}), and luminosity (Eq. \ref{eq:lfrb}) depend mainly on the magnetar's period and magnetic field and the quake energetics and QPO frequency.$
$For the dynamics in the magnetosphere and the compressed striped wind, numerical simulations \citep{Yu2020,Yuan2022ApJ,Mahlmann2022ApJL} have improved the understanding of the related parameters.$
$Therefore, the major uncertainties mainly come from two quake energetic parameters (\epsilon and \zeta), which requires further studies for a better understanding.$
$With physically reasonable values for these two parameters, we find that this model can naturally and self-consistently interpret the observed frequency, pulse width, luminosity and QPO signal of the radio/X-ray event from SGR 1935+2154 and FRB 20191221A.$
$Our model can also naturally explain the observed wide energy span detected in a relative narrow frequency band, such as the broad energy range (4\times10^{36}-8\times10^{39} erg) observed in FRB 121102 at 1.25 GHz \citep{Li2021Natur}, as the frequency does not depend on the flare energy apparently.$
$And in certain parameter space for young magnetars, the frequency is in the optical band, indicating the possibility of producing fast optical transients.$
$In general, QPOs will modulate the FRB light curve. Direct identification of QPOs may require at least around ten individual pulses (n_A\gtrsim10) in one event \citep[e.g.][]{CHIME2022Natur}.$
$For the high-frequency QPOs (\fa\gtrsim1 kHz), the corresponding FRB pulses may overlap with each other, making it difficult to identify from the data.$
$While for the low-frequency QPOs (\fa\sim10 Hz), one may only expect to detect them from magnetars with P\gtrsim10/\fa\sim1 s so that the QPO signal is not contaminated by the magnetar spin.$
$For repeating FRBs, QPOs would affect the waiting time distribution even if only several pulses are presented in the observed individual events.$
$The peak at 3.4 ms in the waiting time distribution of FRB 121102 may be such a case \citep{Li2021Natur}.$
$As only a small portion of energy is dissipated to power FRBs, the majority of the energy will be dissipated into X-rays or carried by the ejecta.$
$The ejecta would further power an afterglow or a nebula \citep[e.g.][]{Waxman2017,Beloborodov2017,Margalit2018,WangLai2020}.$
$As magnetar flares may eject a large fraction of mass \citep[e.g.][]{Granot2006ApJ}, high-energy cosmic rays can also be produced in such afterglows/nebulae.$
$Besides, such a magnetic reconnection process could also take place in neutron star mergers \citep[e.g.][]{Wang2018,Most2022MNRAS} and accreting black holes \citep[e.g.][]{Beloborodov2017ApJ,Sridhar2021MNRAS}, thus we might also expect such reconnection driven transients from neutron star mergers, X-ray binaries, or active galactic nuclei.$
$\vspace{-5pt}$
$\section*{Acknowledgements}$
$JSW thanks J. Kirk, B. Reville, and F. Guo for discussions.$
$JSW acknowledges the support from the Alexander von Humboldt Foundation.$
$XL is supported by NSERC, fundingreference \#CITA 490888-16 and the Jeffrey L. Bishop Fellowship. Research at Perimeter Institute is supported in part by the Government of Canada through the Department of Innovation, Science and Economic Development Canada and theProvince of Ontario through the Ministry of Colleges and Universities.$
$ZGD is supported by the National Key Research and Development Program of China (grant No. 2017YFA0402600), the National SKA Program of China (grant No. 2020SKA0120300), and the National Natural Science Foundation of China (grant No. 11833003).$
$XFW is supported by the National Natural Science Foundation of China (Grant Nos. 11725314, 12041306).$
$\vspace{-5pt}$
$\section*{Data Availability}$
$There are no new data associated with this article.$
$\vspace{-5pt}$
$\bibliographystyle{mnras}$
$\bibliography{ref}$
$\bsp$
$\label{lastpage}$
$\end{document}\end{eqnarray}}$
$\newcommand{\eeq}{\end{eqnarray}}$
$\newcommand{\bq}{B_{\rm QED}}$
$\newcommand{\fa}{f_{A}}$
$\newcommand{\fac}{f_{A,2}}$
$\newcommand{\Gc}{\Gamma_2}$
$\newcommand{\ffrb}{f_{\rm FRB}}$
$\newcommand{\lc}{L_{\rm FRB,43}}$
$\newcommand{\nuc}{\nu_{\rm c}}$
$\newcommand{\ncc}{\nu_{\rm c,9}}$
$\newcommand{\lwc}{L_{\rm w,45}}$
$\newcommand{\muc}{\mu_{33}}$
$\newcommand{\Bc}{B_{ d,15}}$
$\newcommand{\lac}{L_{A,44}}$
$\newcommand{\Bfms}{B_{\rm FMS}}$
$\newcommand{\xl}{\textcolor{red}}$
$\newcommand{\js}{\textcolor{cyan}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{\beq}{\begin{eqnarray}}$
$\newcommand{$\eeq$}{\end{eqnarray}}$
$\newcommand{$\bq$}{B_{\rm QED}}$
$\newcommand{$\fa$}{f_{A}}$
$\newcommand{$\fa$c}{f_{A,2}}$
$\newcommand{$\Gc$}{\Gamma_2}$
$\newcommand{$\ffrb$}{f_{\rm FRB}}$
$\newcommand{$\lc$}{L_{\rm FRB,43}}$
$\newcommand{$\nuc$}{\nu_{\rm c}}$
$\newcommand{$\ncc$}{\nu_{\rm c,9}}$
$\newcommand{$\lwc$}{L_{\rm w,45}}$
$\newcommand{$\muc$}{\mu_{33}}$
$\newcommand{$\Bc$}{B_{ d,15}}$
$\newcommand{$\lac$}{L_{A,44}}$
$\newcommand{$\Bfms$}{B_{\rm FMS}}$
$\newcommand{$\xl$}{\textcolor{red}}$
$\newcommand{$\js$}{\textcolor{cyan}}$
$\begin{document}$
$\begin{CJK*}{UTF8}{gkai}$
$\title[An Intermediate-field FRB model and QPO]{{An Intermediate-field Fast Radio Burst Model and the Quasi-periodic Oscillation}$
$}$
$\author[Wang et al.]{Jie-Shuang Wang$
$^{1}\thanks{E-mail:jiesh.wang@gmail.com},$
$Xinyu Li$
$^{2,3},$
$Zigao Dai$
$^{4,5},$
$Xuefeng Wu$
$^6\^{1}Max-Planck-Institut f\"ur Kernphysik, Saupfercheckweg 1, D-69117 Heidelberg, Germany \^2Canadian Institute for Theoretical Astrophysics, 60 St George St, Toronto, ON M5R 2M8\^3Perimeter Institute for Theoretical Physics, 31 Caroline Street North, Waterloo, Ontario, Canada, N2L 2Y5\^4Department of Astronomy, University of Science and Technology of China, Hefei 230026, China\^5School of Astronomy and Space Science, Nanjing University, Nanjing 210023, China\^6Purple Mountain Observatory, Chinese Academy of Sciences, Nanjing 210023, China$
$}$
$\label{firstpage}$
$\pagerange{\pageref{firstpage}--\pageref{lastpage}}$
$\maketitle$
$\end{CJK*}$
$\begin{abstract}$
$Quasi-periodic oscillation (QPO) signals are discovered in some fast radio bursts (FRBs) such as FRB 20191221A, as well as in the X-ray burst associated with the galactic FRB from SGR 1935+2154.$
$We revisit the intermediate-field FRB model where the radio waves are generated as fast-magnetosonic waves through magnetic reconnection near the light cylinder.$
$The current sheet in the magnetar wind is compressed by a low frequency pulse emitted from the inner magnetosphere to trigger magnetic reconnection.$
$By incorporating the wave dynamics of the magnetosphere, we demonstrate how the FRB frequency, the single pulse width, and luminosity are determined by the period, magnetic field, QPO frequency and quake energetics of the magnetar.$
$We find that this model can naturally and self-consistently interpret the X-ray/radio event from SGR 1935+2154 and the QPO in FRB 20191221A.$
$It can also explain the observed wide energy range of repeating FRBs in a narrow bandwidth.$
$\end{abstract}$
$\begin{keywords}$
$fast radio bursts -- stars: magnetars -- radiation mechanisms: non-thermal -- MHD -- magnetic reconnection$
$\end{keywords}$
$\section{Introduction}$
$In recent years, the study of fast radio bursts (FRBs) has been greatly advanced by the progress in the observations.$
$New detections of the galactic events \citep{CHIME2020Naturmagnetar,Bochenek2020Natur}, burst polarisation \citep[e.g.][]{Michilli2018,Luo2020Natur}, burst morphology \citep[e.g.][]{Pleunis2021ApJ}, source periodicity \citep[e.g.][]{Chime2020NaturPeriod,Pastor-Marazuela2021Natur,CHIME2022Natur}, source activity \citep[e.g.][]{Li2021Natur} and host galaxies \citep[e.g.][]{Bassa2017ApJL,Bhandari2022AJ}, have led to constraints on both progenitor models and radiation mechanisms.$
$However, a complete theoretical understanding of fast radio bursts is still not available and requires more effort \citep[see][for recent reviews]{Zhang2020Natur,Xiao2021SCPMA}.$
$Highly magnetised compact objects are usually involved as the central engine of FRBs, such as magnetars, pulsars, and accreting black holes. Among them, the magnetar scenario has been confirmed by the detection of the galactic event \citep{CHIME2020Naturmagnetar,Bochenek2020Natur,Li2021NatAs,Mereghetti2020ApJL,Ridnaia2021NatAs,Tavani2021NatAs}.$
$Theoretically, the energy resource can come from the internal magnetic energy of magnetars \citep[e.g.][]{Popov2013,Katz2016,Beloborodov2017,Margalit2020ApJL} or the gravitational potential energy when a companion star is involved \citep[e.g.][]{Wang2016,Dai2016,Wang2018,Zhang2020ApJL,Dai2020ApJL,Most2022MNRAS}.$
$Currently, the mainstream FRB radiation mechanisms can be divided into two categories: the near-field (or close-in) and the far-field (or far-away) model.$
$The near-field model, mainly based on the coherent curvature radiation model near the magnetar surface \citep[e.g.][]{Kumar2017,Yang2018,Lu2020MNRAS,WangFY2020ApJ,Yang2020ApJL,WangWY2020ApJ}, can explain the complex temporal behavior of FRBs, while the bunching mechanism for the coherent curvature radiation is unclear.$
$The far-field model incorporates the synchrotron maser emission at the shock front far from the magnetosphere as the coherent radio emission mechanism  \citep[e.g.][]{Lyubarsky2014,Beloborodov2017,Margalit2018,Waxman2017,Metzger2019MNRAS,Beloborodov2020ApJ,Plotnikov2019,Margalit2020ApJL,Wu2020ApJL,Xiao2020ApJL}.$
$This mechanism has been demonstrated using kinetic plasma simulations \citep[e.g.][]{Plotnikov2019,Sironi2021PhRvL}.$
$However, it is difficult to explain the sub-second scale quasi-periodic oscillations (QPOs) from FRBs \citep{CHIME2022Natur,Pastor-Marazuela2022arXiv}.$
$It is also found that neither model can fully explain the X-ray/radio event from SGR 1935+2154 \citep{Wang2020ApJ}.$
$Recently, a new type of FRB model is proposed where an FRB is radiated as fast-magnetosonic (FMS) waves generated from violent magnetic reconnection triggered by a low-frequency pulse (LFP) compressing the current sheet \citep{Lyubarsky2020ApJ}.$
$As the coherent radio emission is produced near the light cylinder, we refer to it as the "intermediate-field" model.$
$Kinetic plasma simulations \citep{Philippov2019,Mahlmann2022ApJL} have successfully demonstrated the emission of coherent FMS waves through this mechanism.$
$It is also found that the high linear polarisation \citep{Lyubarsky2020ApJ} and the downward frequency drifting \citep{Mahlmann2022ApJL} can be well explained.$
$However, the predicted frequency is significantly lower than the observation of the SGR 1935+2154 event \citep{Wang2020ApJ}.$
$In this paper, we revisit the intermediate-field model and propose that the LFP is produced through nonlinear conversion of Alfv\'en waves, which further leads to the generation of QPOs.$
$In section \ref{sec:radiation_injection}, we briefly review the coherent radiation mechanism from magnetic reconnection.$
$In section \ref{sec:FRB_QPO}, we study the injection of energy through Alfv\'en waves, the wave dynamics in the magnetosphere, and the generation of FRBs and QPOs.$
$In section \ref{sec:explainQPO}, we apply our model to observations.$
$The conclusion and discussion are presented in section \ref{sec:conclusion}.$
$Throughout this paper we adopt the shorthand X=X_n\times 10^n to describe the normalization of quantity X in cgs units.$
$\vspace{-5pt}$
$\section{Reconnection Driven Coherent Radio Emission}\label{sec:radiation_injection}$
$Magnetic reconnection is a process of changing the magnetic topology when two oppositely directed magnetic field lines approach each other forming a current sheet at the centre.$
$The onset of magnetic reconnection is triggered when the current sheet becomes tearing or kink unstable and breaks into a self-similar chain of plasmoids extending down to the kinetic scale \citep{Uzdensky2010PhRvL}.$
$Fast magnetic reconnection proceeds when the plasmoids collide and merge into larger islands.$
$During the coalescence of plasmoids, magnetic energy is dissipated and FMS waves are produced.$
$In highly magnetized systems with magnetization parameter \sigma\equiv B^2/4\pi \rho c^2 \gg1, FMS waves have similar dispersion relation as vacuum electromagnetic waves \citep{Thompson1998,Li2019} and can convert to coherent radio waves \citep[e.g.][]{Lyubarsky2019,Philippov2019,Lyubarsky2020ApJ}.$
$\cite{Lyubarsky2020ApJ} proposed that the coherent emission of FRBs are GHz FMS waves generated by the plasmoid collision during magnetic reconnection.$
$In their picture, the current sheets in the magnetar wind are compressed by an external outgoing LFP and become tearing unstable under the perturbation, and magnetic reconnection is  initiated.$
$Coherent FMS waves are produced in the reconnection and can escape as fast radio bursts.$
$Kinetic simulations by \citet{Mahlmann2022ApJL} have confirmed this process and found that FMS waves takes only a small fraction of the reconnection energy.$
$The characteristic frequency of the coherent FMS waves is determined by the size of the plasmoid \lambda'_c. Note we use the primed symbols to mark parameters measured in the co-moving frame of the plasma throughout the paper.$
$In the plasma comoving frame, \omega'_c \approx c/ \lambda'_c.$
$The typical plasmoid size is found to be proportional to the width of the current sheet \lambda'_c=\varsigma a' with \varsigma\sim10-100 \citep{Philippov2019,Lyubarsky2020ApJ}, and the width depends on the Larmor radius a'=\varpi r_{\rm L} with \varpi being a few.$
$\cite{Lyubarsky2020ApJ} gave the expression of the plasmoid size by balancing the pressure and energy,$
$\begin{eqnarray}$
$    \lambda'_c=\sqrt{\frac{8\pi\beta_{\rm rec}\varpi}{e\sigma_T}}\frac{m_ec^2}{B'^{3/2}},$
$\end{eqnarray}$
$where \sigma_T is the Thomson scattering cross section, \beta_{\rm rec}\sim0.1 is the reconnection rate, B'=B/\Gamma is the magnetic field in the comoving frame and \Gamma is the bulk Lorentz factor.$
$Therefore, the characteristic frequency in the observer's frame is,$
$\beq$
$\nu_{\rm c}\equiv\Gamma\omega'_c/2\pi\approx 2.1 \eta_1^{-1} \Gamma^{-1/2} B_{8}^{3/2}~{\rm GHz},\label{Eq:nuc}$
$\eeq$
$where \eta\equiv\varsigma \varpi^{1/2}\beta_{\rm rec}^{1/2}.$
$This expression is confirmed numerically by \cite{Mahlmann2022ApJL} but with a slightly different expression of \eta.$
$\begin{figure}$
$    \centering$
$    \vspace{-5pt}$
$    \includegraphics[width=0.35\textwidth]{QPO_model.pdf}$
$    \caption{The schematic picture of the intermediate-field FRB model (not to scale):$
$    Alfv\'en waves propagate along field lines of R\gtrsim R_0, and become non-linear at R_{\rm m}, which leads to the conversion to an LFP.$
$    (The Alfv\'en waves and LFP are drawn with different styles and unrealistic wavelengths to make their presence clear.)$
$    The LFP compress the current sheet in the striped magnetar wind near the light cylinder.$
$    High-frequency FMS waves are generated by merging islands during the violent magnetic reconnection, which can escape as an FRB.}$
$    \label{fig:magnetar_model}$
$\end{figure}$
$\vspace{-5pt}$
$\section{Intermediate-field FRB models and QPOs}\label{sec:FRB_QPO}$
$\subsection{Energy injection via Alfv\'en waves}$
$For a magnetar with a surface dipole magnetic field B_d and radius R_*\approx10^6 cm, the total magnetic energy is E_{B}=B_d^2R_{*}^3/6\approx1.7\times10^{47}$\Bc$^2 erg.$
$The internal magnetic field B_* can be higher and we parameterize it by B_*= \zeta B_d with \zeta>1.$
$The internal magnetic energy can be released by a sudden starquake and crustal motion which are accompanied by the crustal shear oscillations \citep[e.g.][]{Thompson1995MNRAS,Thompson2001,Duncan1998ApJL}.$
$Such oscillations are proposed to explain the QPOs observed in magnetar X-ray flares \citep[e.g.][]{Israel2005,Strohmayer2005,Strohmayer2006ApJ}.$
$As the shear wave propagates in the crust, it shakes the anchored magnetic field lines, and launches Alfv\'en waves into the magnetosphere \citep{Thompson2001,Li2016,Thompson2017,Bransgrove2020}.$
$The Alfv\'en waves carry energy into the magnetosphere and propagate along the field lines,$
$as sketched in Fig.~\ref{fig:magnetar_model}.$
$The field-line equation is r=R\sin^2{\theta}, where R is the maximum distance of the field line to the magnetar center and \theta is the angle to the magnetic axis.$
$For field lines with R\gg R_*, the field-line length is l\simeq\pi R/2.$
$The generated Alfv\'en waves of frequency $\fa$ in the magnetosphere can propagate along field lines with l\geq l_0=\pi R_0/2=c/$\fa$=1.9\times10^{10}$\fa$^{-1}~{\rm cm} as a perturbation.$
$Note that Alfv\'en waves travelling on field lines with l<l_0 may suffer strong nonlinear interaction and the dissipation mechanism is unclear. Thus we only consider magnetar quakes occurred near the magnetic polar region, so that magnetar oscillations can launch Alfv\'en waves into the magnetosphere before damping \citep[e.g.][]{Levin2007MNRAS,Hoven2012}.$
$We consider a sudden release of internal magnetic energy of E_q=h S B_*^2/8\pi in the crust with a crust thickness h\approx10^5 cm and an area S_q.$
$The magnetar quake can simultaneously launch a number of waves with different frequencies, that are of different damping time-scales \citep[e.g.][]{Levin2007MNRAS,Hoven2012,Gabler2016MNRAS}.$
$We assume that the Alfv\'en wave of frequency f_A is the dominated one, and a number (n_A) of waves at this frequency has been excited.$
$The energy carried by each wave is parameterized as \epsilon E_q\approx L_A/f_A with a power L_A and \epsilon<1/n_A.$
$The wave is launched from the magnetar surface within the same area,$
$\begin{eqnarray}$
$    S_q=2.5\times10^8$\lac$$\fa$c^{-1}$\Bc$^{-2}\epsilon^{-1}\zeta^{-2} {\rm cm}^2.$
$\end{eqnarray}$
$The corresponding wave amplitude in the magnetosphere is {\delta B}=\sqrt{8\pi L_A/S_qc}.$
$Its relative amplitude to the background field near the magnetar surface is$
$\begin{eqnarray}$
$    \frac{\delta B}{B_d}\approx 0.02$\fa$c^{1/2}\epsilon^{1/2}\zeta,$
$\end{eqnarray}$
$As the Alfv\'en wave propagates in the magnetosphere, its amplitude grows as {\delta B}/{B_d}\propto r^{3/2} \citep[e.g.][]{Kumar2020,Yuan2020}.$
$When the relative amplitude grows to {\delta B}/{ B}\approx1 \citep{Yuan2020,Yuan2022ApJ} at a distance of$
$\begin{eqnarray}$
$   R_{\rm m}\approx 1.4\times 10^7$\fa$c^{-1/3}\epsilon^{-1/3}\zeta^{-2/3}~\rm{cm},$
$\end{eqnarray}$
$the Alfv\'en waves will become nonlinear and shear the magnetosphere.$
$The background magnetic field here can be estimated from a dipole B_{\rm m}=B_*R_*^3R_{\rm m}^{-3}.$
$Under the strong shear, the magnetosphere becomes unstable.$
$A large part of the Alfv\'en waves will be converted to FMS waves that propagates outwards radially in the form of a LFP \citep{Yuan2021} due to the interaction with the curved magnetic field line.$
$The remaining Alfv\'en waves will dissipate inside the magnetosphere by nonlinear instabilities \citep{Yuan2020} or absorbed by the magnetar \citep{Li2015}.$
$Self interactions among multiple Alfv\'en waves can enhance the wave dissipation through turbulence cascade, increasing fast wave conversion rate \citep{Li2019} or leading to fast dissipation \citep{Li2021}.$
$\vspace{-5pt}$
$\subsection{Generation of FRBs}$
$An illustration for the generation of FRBs is shown in Fig. \ref{fig:magnetar_model}.$
$During the propagation of the LFP at r>R_{\rm m}, its toroidal field follows rB_{\rm LFP } = B_{\rm LFP, m}R_{\rm m} with B_{\rm LFP, m}\approx B_{\rm m} \citep{Yuan2020,Yuan2022ApJ}.$
$As the energy contained in the LFP is higher than that of background field, the magnetosphere is distorted.$
$The poloidal field follows r^2 B_p= B_{\rm m}R_{\rm m}^{2} from the conservation of magnetic flux.$
$Thus the magnetosphere at R>R_{\rm m} can be treated as a split monopole \citep{Parfrey2013,Yuan2020,Yuan2022ApJ}.$
$When the LFP propagates beyond the light cylinder at R_L=c P/2\pi, where P is the magnetar rotation period, it enters the magnetar wind zone.$
$The toroidal field of the LFP and the poloidal field of the magnetosphere at the light cylinder are$
$\begin{eqnarray}$
$    B_{\rm LFP, L}=3.4\times10^8$\Bc$$\fa$c^{2/3}\epsilon^{2/3}\zeta^{4/3}\left(\frac{P}{3 {\rm s}}\right)^{-1} \rm{G}\label{eq:bphi_LC};\B_{p,m}(R_{\rm L})=3.4\times10^5$\Bc$$\fa$c^{1/3}\epsilon^{1/3}\zeta^{2/3}\left(\frac{P}{3 {\rm s}}\right)^{-2} \rm{G}.\label{eq:bpol_LC}$
$\end{eqnarray}$
$The LFP compresses the striped wind and accelerates the plasma to a bulk Lorentz factor of \Gamma\approx0.5\sqrt{B_{\rm LFP, L}/B_{\rm w}(R_{\rm L}) }, where the wind field is described as a monopole with B_{\rm w}(r)=B_{p}(R_L)R_L/r and B_{p}(R_L) is the poloidal field of the magnetosphere at the light cylinder.$
$The pulse front propagates into the magnetar wind launched by the unperturbed magnetosphere with B_{p}(R_L)=B_*R_*^3R_{\rm L}^{-3}.$
$However, the energy carried by the LFP amplifies the magnetic field at the light cylinder to B_{p,m}(R_{\rm L}) \citep{Yuan2020,Yuan2022ApJ}, which further enhances the toroidal magnetic field in the striped wind to B_{\rm w}(R_{\rm L})\sim B_{p,m}(R_{\rm L}).$
$Therefore, the rear part of LFP will interact with the current sheets in the striped wind launched by the perturbed magnetosphere (see. Fig. \ref{fig:magnetar_model}).$
$In reality, the amplification of the wind toroidal field is highly dynamic, we therefore expect the wind field to be in the range B_{\rm w}(R_{\rm L})\in[B_*R_*^3R_{\rm L}^{-3},\; B_{p,m}(R_{\rm L})].$
$The corresponding maximum and minimum Lorentz factors of the accelerated LFPs are then$
$\begin{eqnarray}$
$    &&\Gamma^{\rm max}=496 \epsilon^{1/3}\zeta^{2/3}$\fa$c^{1/3}\left(\frac{P}{3 {\rm s}}\right);\\\nonumber$
$    &&\Gamma^{\rm min}=16 \epsilon^{1/6}\zeta^{1/3}$\fa$c^{1/6}\left(\frac{P}{3 {\rm s}}\right)^{1/2}.\label{eq.Gamma}$
$\end{eqnarray}$
$In the following we parameterize \Gamma=100$\Gc$ which lies between \Gamma^{\rm min} and \Gamma^{\rm max}.$
$And we expect \Gamma\rightarrow\Gamma^{\rm min} for LFPs consisting of multiple waves (e.g. n_A\gtrsim3), such as in the SGR 1935+2154 event and FRB 20191221A (see Sec. \ref{sec:explainQPO} for more details).$
$The LFP compresses the current sheet in the striped wind, which triggers magnetic reconnection and leads to the generation of high-frequency FMS waves \citep{Lyubarsky2020ApJ,Mahlmann2022ApJL}.$
$Substituting B_{\rm LFP,L} and \Gamma into Eq.~(\ref{Eq:nuc}), we obtain the frequency of the FMS waves,$
$\begin{eqnarray}$
$    $\nuc$=1.3$\fa$c$\Bc$^{3/2}\epsilon\zeta^2\left(\frac{P}{3 {\rm s}}\right)^{-3/2}\eta_1^{-1}$\Gc$^{-1/2} \rm{GHz},\label{eq:nu_final}$
$\end{eqnarray}$
$which lies in the radio band.$
$The produced FMS waves suffers from nonlinear interactions in the magnetosphere that may prevent it from escaping.$
$We can estimate the optical depth of the nonlinear interactions for the FMS waves following \citet{Lyubarsky2020ApJ},$
$\begin{eqnarray}$
$    \tau_{\rm NL}\sim \frac{\kappa 2\pi\nu R_{\rm L}}{2\Gamma^2c}=0.5\kappa_{-5.5}$\ncc$\left(\frac{P}{3 {\rm s}}\right)$\Gc$^{-2},$
$\end{eqnarray}$
$where \kappa=$\Bfms$^2/B_{\rm LFP, L}^2 is the energy ratio of the high-frequency FMS waves to the LFP. In 2D simulations, it is found that a fraction \kappa\sim10^{-4} of energy from the LFP is converted to the FMS, interpreted as an upper limit of the efficiency \citep{Mahlmann2022ApJL}.$
$The ratio of radio to X-ray energy of FRB200428 is found to be 10^{-6}-10^{-5} \citep{Ridnaia2021NatAs}.$
$Here we take \kappa\sim10^{-5.5} to be the fiducial value.$
$Therefore, the FMS waves generated at the light cylinder can escape the magnetosphere without significant dissipation and be observed as FRBs.$
$There are also current sheets behind the LFP inside the magnetosphere, which has been suggested to produce an FRB \citep{Wang2020ApJ,Yuan2020,Yuan2022ApJ}.$
$However, we would argue below that the FMS waves produced behind the LFP cannot be the primary source of FRB.$
$On one hand, FMS waves emitted deep inside the magnetosphere suffers from strong dissipation due to the nonlinear interactions.$
$The optical depth inside the magnetosphere is \tau_{\rm NL}\approx({$\Bfms$}/{B_p})^2\omega R/c \citep[see Eq. A3 in][]{Lyubarsky2020ApJ} with the FMS wave magnetic field {$\Bfms$}=\sqrt{2L_{\rm FMS}/ r^2} and B_p being treated as a monopole.$
$Setting \tau_{\rm NL}\sim10, we obtain a mean-free-path$
$\begin{eqnarray}$
$\ell\sim6.1\times10^7$\Bc$^{1/2}$\fa$c^{1/6}\epsilon^{1/6}\zeta^{1/3}$\ncc$^{-1/4}L_{\rm FMS,42}^{-1/4} {\rm cm},$
$\end{eqnarray}$
$which is much smaller than R_L for a typical magnetar with period 0.1-10~s.$
$On the other hand, the reconnection of the current sheet behind the LFP is supported by the reversed poloidal magnetic field \citep{Yuan2022ApJ}.$
$Near the light cylinder where the produced FMS can escape, we can estimate$
$\begin{eqnarray}$
$    \frac{B_{p,m}^2(R_{\rm L})}{B_{\rm LFP, L}^2}=1.0\times10^{-6}$\fa$c^{-2/3}\epsilon^{-2/3}\zeta^{-4/3}\left(\frac{P}{3 {\rm s}}\right)^{-2}.$
$\end{eqnarray}$
$The magnetic energy here is much smaller than the energy of the LFP, therefore the escaping FMS wave must be dominated by those generated in the striped wind.$
$\vspace{-5pt}$
$\subsection{QPOs in FRBs}$
$The LFP is made up of low-frequency FMS waves converted from the propagating Alfv\'en waves.$
$This process can be viewed as the nonlinear interaction between Alfv\'en waves and the curved background magnetic field.$
$The leading order wave interaction is the three-wave interaction with the frequency of FMS wave given by f_{\rm LFP}=f_A+f_{\rm bg}.$
$As the background magnetic field can be viewed as a large scale perturbation, i.e. f_{\rm bg}\approx 0, the resulting LFP wave will have the same frequency as the Alfv\'en wave f_{\rm LFP}\approx $\fa$.$
$The LFP propagates outward radially with its thickness conserved as found by numerical simulations \citep{Yuan2022ApJ}. Therefore the LFP frequency remains constant.$
$The energy dissipation of the magnetic reconnection at the light cylinder will be mediated by this LFP wave frequency, so that we expect$
$\begin{eqnarray}$
$    $\ffrb$\sim f_{\rm LFP}\approx$\fa$.$
$\end{eqnarray}$
$Note in this expression, we assume $\fa$>1/P, which will in general be satisfied for magnetars with P\gtrsim0.1 s. Otherwise the FRB profile will also be modulated by the spin period.$
$The LFP accelerates during the propagation to the light cylinder.$
$Therefore the observed width of one single pulse and peak luminosity is$
$\begin{eqnarray}$
$    W_{\rm p}\approx1/(\Gamma$\fa$),\label{eq:tpulse}\L_{\rm FMS}\approx\kappa L_A \Gamma\label{eq:lfrb}.$
$\end{eqnarray}$
$The frequency of Alfv\'en waves depends on the magnetar properties.$
$\cite{Duncan1998ApJL} studied the shear wave in the magnetar crust and found that the frequency of toroidal modes with the lowest radial node (symbolized by _lt_0) follows $\fa$(_lt_0)\approx12.2l^{1/2}(l+1)^{1/2} \rm{Hz} with l\geq2 being an integer \citep{Duncan1998ApJL}.$
$Later it is found that the crustal shear modes would couple with the MHD modes of the magnetar core, and the frequency and duration of the oscillation will be modified \citep[e.g.][]{Levin2007MNRAS,Hoven2012,Gabler2016MNRAS}.$
$Such theories indicate that the fundamental frequency of magnetar QPOs can be as low as a few Hz, and it can be much higher for high order overtones, even up to a few kHz \citep[e.g.][]{Hoven2012}.$
$The observed QPO signals in FRBs will be dominated by one or several major frequencies which carries most of the energy, and the FRB light curve is made by their superposition.$
$\vspace{-5pt}$
$\section{Application to the observed QPOs and the SGR 1935+2154 event}\label{sec:explainQPO}$
$Observations show that the FRB from the galactic magnetar, SGR 1935+2154 \citep{CHIME2020Naturmagnetar,Bochenek2020Natur}, is associated with a hard X-ray burst by several X-ray instruments\citep{Li2021NatAs,Mereghetti2020ApJL,Ridnaia2021NatAs,Tavani2021NatAs}.$
$This magnetar is of spin period and surface magnetic field P=3.2 s and B_*=2.2\times10^{14} G respectively \citep{Olausen2014,Israel2016}.$
$Two radio pulses separated by 30 ms are found in this FRB, while more pulses are detected in X-ray bands.$
$Especially, a QPO of $\fa$\approx40 Hz and n_A\gtrsim8 is detected in the X-ray light curve by the Insight-HXMT \citep{Li2022ApJ}.$
$The observed pulse width is 0.3-0.6 ms ms by CHIME and STARE2 in 0.6-1.3 GHz \citep{CHIME2020Naturmagnetar,Bochenek2020Natur}.$
$Substituting \epsilon<1/n_A\sim1/8, W_{\rm p}\sim0.5 ms, and  $\nuc$\sim1 GHz as well as the observed radio peak luminosity L_{\rm FMS}\approx10^{38} erg/s \citep{Bochenek2020Natur,Zhou2020ApJ} into Eqs. (\ref{eq:nu_final}, \ref{eq:tpulse} and \ref{eq:lfrb}), we obtain \Gamma\sim50, \zeta\gtrsim11\eta_1^{1/2} and L_A\approx 6\times10^{41} \kappa_{-5.5}^{-1} erg/s.$
$The required \Gamma is in the allowed range [22\eta_1^{1/6}, 977\eta_1^{1/3}] from Eq. (\ref{eq.Gamma}).$
$And the required L_A is higher than the observed X-ray luminosity, 1.1\times10^{41}~erg/s \citep{Ridnaia2021NatAs}.$
$Therefore, our model can explain this event self-consistently.$
$QPO signals are also observed in individual FRB events, such as the 4.6 Hz QPO in FRB 20191221A, 357.1 Hz in FRB 20210206A, 93.4 Hz in FRB 20210213A detected by CHIME \citep{CHIME2022Natur}, and the 2.4 kHz QPO in FRB 20201020A detected by Apertif \citep{Pastor-Marazuela2022arXiv}.$
$The observed QPOs with frequencies larger than 93.4 Hz mentioned above can be explained by the overtones in a wide parameter range. While the 4.6 Hz QPO in FRB 20191221A may relate to the fundamental mode. We here focus on this source, which has the highest significance \citep{CHIME2022Natur}.$
$The observed radio frequency, single pulse width, and QPO frequency of FRB 20191221A are $\nuc$\approx0.7 GHz, W_{\rm p}\approx4 ms, and f_A\approx4.6 Hz with n_A\approx9, respectively.$
$The total duration of this event is \sim3 s and we would expect P>3 s so that the QPO signal is not contaminated by the magnetar rotation.$
$Substituting them into Eqs. (\ref{eq:nu_final} and \ref{eq:tpulse}), we obtain \Gamma=54 and \zeta\gtrsim9\eta_1^{1/2}(P/3 {\rm s})^{3/4}$\Bc$^{-3/4}.$
$The required \Gamma is in the allowed range [14\eta_1^{1/6}(P/3 {\rm s})^{3/4}$\Bc$^{-1/4},367\eta_1^{1/3}(P/3 {\rm s})^{3/2}$\Bc$^{-1/2}].$
$Thus the QPO in FRB 20191221A can also be explained by our model.$
$\vspace{-5pt}$
$\section{Conclusion and Discussion}\label{sec:conclusion}$
$In this paper we revisited the intermediate-field FRB model and studied the QPOs.$
$The launch of Alfv\'en waves into the magnetosphere generates an LFP, which is made up of FMS waves at approximately the same frequencies of Alfv\'en waves ($\fa$).$
$This further generates FRBs when the LFP dissipates its energy at the current sheet near the light cylinder.$
$The FRB light curve is thus modulated by the frequency of the LFP, exhibiting QPOs at a frequency \sim$\fa$.$
$The major difference of our calculations from \cite{Lyubarsky2020ApJ} is the treatment of magnetic field configurations of the LFP and the magnetar wind, as we considered the wave dynamics in the magnetosphere.$
$The FRB frequency (Eq. \ref{eq:nu_final}), the single pulse width (Eq. \ref{eq:tpulse}), and luminosity (Eq. \ref{eq:lfrb}) depend mainly on the magnetar's period and magnetic field and the quake energetics and QPO frequency.$
$For the dynamics in the magnetosphere and the compressed striped wind, numerical simulations \citep{Yu2020,Yuan2022ApJ,Mahlmann2022ApJL} have improved the understanding of the related parameters.$
$Therefore, the major uncertainties mainly come from two quake energetic parameters (\epsilon and \zeta), which requires further studies for a better understanding.$
$With physically reasonable values for these two parameters, we find that this model can naturally and self-consistently interpret the observed frequency, pulse width, luminosity and QPO signal of the radio/X-ray event from SGR 1935+2154 and FRB 20191221A.$
$Our model can also naturally explain the observed wide energy span detected in a relative narrow frequency band, such as the broad energy range (4\times10^{36}-8\times10^{39} erg) observed in FRB 121102 at 1.25 GHz \citep{Li2021Natur}, as the frequency does not depend on the flare energy apparently.$
$And in certain parameter space for young magnetars, the frequency is in the optical band, indicating the possibility of producing fast optical transients.$
$In general, QPOs will modulate the FRB light curve. Direct identification of QPOs may require at least around ten individual pulses (n_A\gtrsim10) in one event \citep[e.g.][]{CHIME2022Natur}.$
$For the high-frequency QPOs ($\fa$\gtrsim1 kHz), the corresponding FRB pulses may overlap with each other, making it difficult to identify from the data.$
$While for the low-frequency QPOs ($\fa$\sim10 Hz), one may only expect to detect them from magnetars with P\gtrsim10/$\fa$\sim1 s so that the QPO signal is not contaminated by the magnetar spin.$
$For repeating FRBs, QPOs would affect the waiting time distribution even if only several pulses are presented in the observed individual events.$
$The peak at 3.4 ms in the waiting time distribution of FRB 121102 may be such a case \citep{Li2021Natur}.$
$As only a small portion of energy is dissipated to power FRBs, the majority of the energy will be dissipated into X-rays or carried by the ejecta.$
$The ejecta would further power an afterglow or a nebula \citep[e.g.][]{Waxman2017,Beloborodov2017,Margalit2018,WangLai2020}.$
$As magnetar flares may eject a large fraction of mass \citep[e.g.][]{Granot2006ApJ}, high-energy cosmic rays can also be produced in such afterglows/nebulae.$
$Besides, such a magnetic reconnection process could also take place in neutron star mergers \citep[e.g.][]{Wang2018,Most2022MNRAS} and accreting black holes \citep[e.g.][]{Beloborodov2017ApJ,Sridhar2021MNRAS}, thus we might also expect such reconnection driven transients from neutron star mergers, X-ray binaries, or active galactic nuclei.$
$\vspace{-5pt}$
$\section*{Acknowledgements}$
$JSW thanks J. Kirk, B. Reville, and F. Guo for discussions.$
$JSW acknowledges the support from the Alexander von Humboldt Foundation.$
$XL is supported by NSERC, fundingreference \#CITA 490888-16 and the Jeffrey L. Bishop Fellowship. Research at Perimeter Institute is supported in part by the Government of Canada through the Department of Innovation, Science and Economic Development Canada and theProvince of Ontario through the Ministry of Colleges and Universities.$
$ZGD is supported by the National Key Research and Development Program of China (grant No. 2017YFA0402600), the National SKA Program of China (grant No. 2020SKA0120300), and the National Natural Science Foundation of China (grant No. 11833003).$
$XFW is supported by the National Natural Science Foundation of China (Grant Nos. 11725314, 12041306).$
$\vspace{-5pt}$
$\section*{Data Availability}$
$There are no new data associated with this article.$
$\vspace{-5pt}$
$\bibliographystyle{mnras}$
$\bibliography{ref}$
$\bsp$
$\label{lastpage}$
$\end{document}\end{eqnarray}}$
$\newcommand{$\eeq$}{\end{eqnarray}}$
$\newcommand{$\bq$}{B_{\rm QED}}$
$\newcommand{$\fa$}{f_{A}}$
$\newcommand{$\fa$c}{f_{A,2}}$
$\newcommand{$\Gc$}{\Gamma_2}$
$\newcommand{$\ffrb$}{f_{\rm FRB}}$
$\newcommand{$\lc$}{L_{\rm FRB,43}}$
$\newcommand{$\nuc$}{\nu_{\rm c}}$
$\newcommand{$\ncc$}{\nu_{\rm c,9}}$
$\newcommand{$\lwc$}{L_{\rm w,45}}$
$\newcommand{$\muc$}{\mu_{33}}$
$\newcommand{$\Bc$}{B_{ d,15}}$
$\newcommand{$\lac$}{L_{A,44}}$
$\newcommand{$\Bfms$}{B_{\rm FMS}}$
$\newcommand{$\xl$}{\textcolor{red}}$
$\newcommand{$\js$}{\textcolor{cyan}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# An Intermediate-field Fast Radio Burst Model and the Quasi-periodic Oscillation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2210.09930-b31b1b.svg)](https://arxiv.org/abs/2210.09930)<mark>Appeared on: 2022-10-19</mark> - _5 pages, 1 figure, submitted. Comments welcome!_

</div>
<div id="authors">

Jie-Shuang Wang, <mark><mark>Xinyu Li</mark></mark>, Zigao Dai, Xuefeng Wu

</div>
<div id="abstract">

**Abstract:** Quasi-periodic oscillation (QPO) signals are discovered in some fast radio bursts (FRBs) such as FRB 20191221A, as well as in the X-ray burst associated with the galactic FRB from SGR 1935+2154.We revisit the intermediate-field FRB model where the radio waves are generated as fast-magnetosonic waves through magnetic reconnection near the light cylinder.The current sheet in the magnetar wind is compressed by a low frequency pulse emitted from the inner magnetosphere to trigger magnetic reconnection.By incorporating the wave dynamics of the magnetosphere, we demonstrate how the FRB frequency, the single pulse width, and luminosity are determined by the period, magnetic field, QPO frequency and quake energetics of the magnetar.We find that this model can naturally and self-consistently interpret the X-ray/radio event from SGR 1935+2154 and the QPO in FRB 20191221A.It can also explain the observed wide energy range of repeating FRBs in a narrow bandwidth.

</div>

<div id="div_fig1">

<img src="tmp_2210.09930/./QPO_model.png" alt="Fig1" width="100%"/>

**Figure 1. -** The schematic picture of the intermediate-field FRB model (not to scale):
    Alfv\'en waves propagate along field lines of $R\gtrsim R_0$, and become non-linear at $R_{\rm m}$, which leads to the conversion to an LFP.
    (The Alfv\'en waves and LFP are drawn with different styles and unrealistic wavelengths to make their presence clear.)
    The LFP compress the current sheet in the striped magnetar wind near the light cylinder.
    High-frequency FMS waves are generated by merging islands during the violent magnetic reconnection, which can escape as an FRB. (*fig:magnetar_model*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

68  publications in the last 7 days.
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers